In [14]:
from serpapi import GoogleSearch
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import os
from functions import *

In [15]:
coin = 'ETH'
queries = None
with open('queries.txt') as queries:
    query = queries.read().splitlines()
    query = [f'{coin} {q}' for q in query]
    queries = query

In [12]:
PATH = f'{coin}_newspapers.csv'
newspapers = pd.DataFrame()
if(os.path.exists(PATH)):
    newspapers = pd.read_csv(PATH)

for q in tqdm(queries):
    stuff = get_newspapers(q)
    newspapers = pd.concat([newspapers, stuff], ignore_index=True)

newspapers.to_csv(PATH)

100%|██████████| 3/3 [30:53<00:00, 617.75s/it]


  1%|          | 1/98 [00:05<09:17,  5.74s/it]


KeyboardInterrupt: 